# Glue ETL Transformation - Handling Missing Fields and Null Values

Considering this json schema:

```python
{"id": "8ec3", "firstname": "Bond", "lastname": "James"}
```

Consider these two common situatesion:

1. field not exists:

```python
{"id": "8ec3", "firstname": "Bond"}
```

2. value is null:

```python
{"id": "8ec3", "firstname": None, "lastname": "James"}
```

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pprint import pprint

# Create SparkContext
sparkContext = SparkContext.getOrCreate()
# Create Glue Context
glueContext = GlueContext(sparkContext)
# Get spark session
spark = glueContext.spark_session

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1646085135716_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
class Config:
    bucket = "aws-data-lab-sanhe-for-everything-us-east-2"
    prefix = "poc/learn-big-data-on-aws/glue-job-examples/missing-fields-and-null-values"
    n_files = 3
    n_records_per_file = 100
    
    @property
    def s3path_prefix(self):
        return S3Path(self.bucket, self.prefix)
    
config = Config()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# gdf = Glue Dynamic Frame
gdf = glueContext.create_dynamic_frame.from_options(
    connection_type="s3", 
    connection_options=dict(
        paths=[
            f"s3://{config.bucket}/{config.prefix}/"
        ],
        recurse=True,
    ),
    format="json",
    format_options=dict(multiLine=True),
    transformation_ctx="datasource",
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# print data schema
# Glue Dynamic Frame 足够聪明, 能够自动根据统计数据推断出 Schema
gdf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- id: int
|-- with_missing_field: int
|-- with_null_value: int

In [4]:
# preview the data
# 在读取的过程中, 如果 field 不存在, 就已经被读成 null value 了
gdf.toDF().show(40, truncate=False, vertical=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+---------------+
|id |with_missing_field|with_null_value|
+---+------------------+---------------+
|20 |83                |65             |
|40 |null              |7              |
|60 |83                |46             |
|80 |29                |56             |
|100|79                |73             |
|108|79                |40             |
|128|38                |13             |
|148|88                |44             |
|168|8                 |30             |
|188|null              |64             |
|209|94                |46             |
|229|62                |69             |
|249|14                |71             |
|269|42                |27             |
|289|45                |39             |
|1  |83                |37             |
|21 |94                |71             |
|41 |46                |61             |
|61 |46                |27             |
|81 |43                |52             |
|109|72                |82             |
|129|60         

In [5]:
# count rows
gdf.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

300

## Map transformation operator with missing field and bad field

对每一行按照一定逻辑进行 transformation 是非常常见的操作. 在 Glue ETL 中对应的 transformation operator 是 ``Map`` + UDF (User defined function). ``Map`` 的操作对象是 Row. **而且是具有容错的**. 如果许多列中只有部分列出错了, 那么在结果中就不会包含这些出错的列. 你可以用 ``stageThreshold``, ``totalThreshold`` 这两个参数控制是否抛出异常.


In [6]:
# Map transformation operator is fault tolerant.
# you can use ``stageThreshold``, ``totalThreshold`` to raise exception of error rate >= threshold
# ref: https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-transforms-map.html
def times_100(row):
    row["with_missing_field"] = row["with_missing_field"] * 100
    row["with_null_value"] = row["with_null_value"] * 100
    return row

gdf_tmp = Map.apply(frame=gdf, f=times_100)
gdf_tmp.toDF().show(40, truncate=False, vertical=False)
gdf_tmp.count() # will be less than gdf.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+---+
|with_missing_field|with_null_value|id |
+------------------+---------------+---+
|8300              |6500           |20 |
|8300              |4600           |60 |
|2900              |5600           |80 |
|7900              |7300           |100|
|7900              |4000           |108|
|3800              |1300           |128|
|8800              |4400           |148|
|800               |3000           |168|
|9400              |4600           |209|
|6200              |6900           |229|
|1400              |7100           |249|
|4200              |2700           |269|
|4500              |3900           |289|
|8300              |3700           |1  |
|9400              |7100           |21 |
|4600              |6100           |41 |
|4600              |2700           |61 |
|4300              |5200           |81 |
|7200              |8200           |109|
|9400              |8900           |149|
|4000              |9800           |169|
|7100           

在 PySpark 中, 你无法用 UDF 对整个 Row 进行操作, 你只能对每个 Column 使用

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

pdf = gdf.toDF()

udf_times_100 = udf(
    lambda x: x * 100, 
    IntegerType(),
)

pdf_tmp = pdf.dropna(subset=["with_missing_field", "with_null_value"]) \
    .select(
        pdf.id, 
        udf_times_100(pdf.with_missing_field).alias("with_missing_field"),
        udf_times_100(pdf.with_null_value).alias("with_null_value"),
    )
pdf_tmp.show(5, truncate=False, vertical=False)
pdf_tmp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+---------------+
|id |with_missing_field|with_null_value|
+---+------------------+---------------+
|20 |8300              |8300           |
|60 |8300              |8300           |
|80 |2900              |2900           |
|100|7900              |7900           |
|108|7900              |7900           |
+---+------------------+---------------+
only showing top 5 rows

239

In [24]:
udf_add_then_times_100 = udf(
    lambda x, y: (x + y) * 100, 
    IntegerType(),
)

pdf_clean = pdf.dropna(subset=["with_missing_field", "with_null_value"])
pdf_tmp = pdf_clean.select(
    udf_add_then_times_100(
        pdf_clean.with_missing_field, 
        pdf_clean.with_null_value,
    )
)
pdf_tmp.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------+
|<lambda>(with_missing_field, with_null_value)|
+---------------------------------------------+
|                                        14800|
|                                        12900|
|                                         8500|
|                                        15200|
|                                        11900|
+---------------------------------------------+
only showing top 5 rows